# IMDB Movie Cleaning
![ImdbIcon](images/imdbheader.jpg)

### Notebook Overview

For my capstone project, I want to see if I can accurately predict a movies score based on the actors, actresses, and directors. I also want to be able to predict how well a movie would do in the future based off of the roles that a user chooses. For example, one might be able to see how well Leonardo Dicaprio and Anne Hathaway would do in a comedy movie, or any movie of their choosing. It should be known that some actors and actresses may be one hit wonders, and there are actors/actresses not in this list that will have successful careers, but with the data I have chosen to work with, we will not account for those factors.

In this notebook I will be reading in four seperate datasets and merging them together based off of specific id columns. I remove columns with too many null values to work with and impute a very small amount of values (mean for 18 rows). I fill some nan values with unknown such as the plot and director where necessary so that I don't remove too many roww of valuable data. The location is very important, and movies outside the USA have too many null values so I chose to only work with USA movies. I am only focusing on specific roles such as actors, actresses, director, producer, and writer, so I removed those that are not as important for this specific project. I also took out all individuals that have passed away so that I do not have a recommended role for someone that has passed away. I also took only movies in the past 20 years so that I do not have recommendations for people that may have passed away similar to date of death column. I also split the genre columns because some movies have multiple genres, and by splitting, I can account for a persons impact in the specific genre instead of removing.

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Import Data

In [2]:
movies = pd.read_csv('./data/IMDb_movies.csv')
names = pd.read_csv('./data/IMDb_names.csv')
ratings = pd.read_csv('./data/IMDb_ratings.csv')
titles = pd.read_csv('./data/IMDb_title_principals.csv')
budgets_df = pd.read_csv('./data/tmdb_movies_data.csv')

C:\Users\nolan_fur2pfn\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\nolan_fur2pfn\.conda\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merge Data

In [3]:
# Merging movies with ratings
movies = movies.merge(ratings, left_on = 'imdb_title_id', right_on = 'imdb_title_id')
# Merging movives with tiles
movies = movies.merge(titles, left_on = 'imdb_title_id', right_on = 'imdb_title_id')
# Merging movies with names (actors/actresses/directors)
movies = movies.merge(names, left_on = 'imdb_name_id', right_on = 'imdb_name_id')

### Dropping Columns

In [4]:
# Dropping columns that serve no values
movies.drop(columns = ['imdb_title_id', 'title', 'language', 'production_company', 'budget', 'metascore', 'reviews_from_critics', 
                       'usa_gross_income', 'worlwide_gross_income', 'allgenders_0age_avg_vote', 'allgenders_0age_votes', 
                       'allgenders_18age_avg_vote','allgenders_18age_votes', 'males_0age_avg_vote', 'males_0age_votes',
                       'males_18age_avg_vote', 'males_18age_votes', 'females_0age_avg_vote', 'females_0age_votes', 
                       'females_18age_avg_vote', 'females_18age_votes', 'females_30age_avg_vote', 'females_30age_votes', 
                       'females_45age_avg_vote', 'females_45age_votes', 'job', 'characters', 'imdb_name_id', 'birth_name',
                       'reviews_from_users', 'writer', 'allgenders_30age_avg_vote', 'allgenders_30age_votes', 
                       'allgenders_45age_avg_vote', 'allgenders_45age_votes', 'males_allages_avg_vote', 'males_allages_votes', 
                       'males_30age_avg_vote',  'males_30age_votes', 'males_45age_avg_vote', 'males_45age_votes', 
                       'females_allages_avg_vote', 'females_allages_votes', 'top1000_voters_rating', 'top1000_voters_votes', 
                       'date_published', 'ordering', 'director', 'non_us_voters_rating', 'non_us_voters_votes'], inplace=True)

In [5]:
budgets_df = budgets_df[['original_title', 'popularity', 'budget', 'revenue', 'tagline', 'budget_adj', 'revenue_adj']]

In [6]:
movies = movies.merge(budgets_df, left_on = 'original_title', right_on = 'original_title')

### Cleaning

In [7]:
# Removing columns that were not movies
movies = movies[movies['year'] != 'TV Movie 2019']
# Movies outside the United States had a lot of missing data.
movies = movies[movies['country'] == 'USA']
# Removing both Reality-TV 
movies = movies[(movies['genre'] != 'Reality-TV') & (movies['genre'] != 'News')]

In [8]:
# A small amount of movies had no description, so I could fill those with Unknown to keep valuable data.
movies['description'] = movies['description'].fillna("Unknown")

In [9]:
# Less than twenty rows were missing voters rating, so I felt comfortable imputing the mean.
movies['us_voters_rating'].fillna((movies['us_voters_rating'].mean()), inplace=True)
# Less than twenty rows were missing voters votes, so I felt comfortable imputing the mean.
movies['us_voters_votes'].fillna((movies['us_voters_votes'].mean()), inplace=True)

In [10]:
# Only looking to work with actors, actresses, and directors.
jobs_list = ['actor', 'actress', 'director']

movies = movies[movies['category'].isin(jobs_list)]

In [11]:
movies.reset_index(drop=True, inplace=True)

In [12]:
# Removing date_of_death and date_of_birth, no longer necessary
movies.drop(columns = ['date_of_death', 'date_of_birth'], inplace=True)

In [13]:
# Converting column year to integer
movies['year'] = movies['year'].astype('int64')

In [14]:
# Renaming original title, category, and description.
movies.rename(columns = {'original_title':'movie_title',
                          'category':'role',
                         'description':'plot'}, inplace=True)

### Dummify Genre

In [15]:
# Using str.get_dummes(",", I can have multiple values in dummy columns)
# For genre, if a movie is Horror AND Action, a 1 is placed in both of those columns
genre_dummies = movies['genre'].str.get_dummies(", ")
# Merging the dummified columns back to the movie dataframe
movies = pd.merge(movies, genre_dummies, left_index =True, right_index=True)
# Dropping genre and country. Only USA movies and genre is now dummified.
movies.drop(columns = ['genre', 'country'], inplace=True)

In [16]:
# Dummify roles (actors, actresses, directors)
movies = pd.get_dummies(movies, columns = ['role'])

In [17]:
# Grouping by movie_title and dividing by the amount of times it is its own row (because of actors, actresses, and directors)
movies_one = movies.groupby(['movie_title']).sum() / movies.groupby(['movie_title']).count()
movies_one = movies_one.reset_index()

In [18]:
# Getting the average score for each name in the dataframe, then changing the column to 'average_role_score'
movies_one = movies.groupby(['name']).sum() / movies.groupby(['name']).count()
movies_one = movies_one.reset_index()
movies_one = movies_one.round(3)
movies_one = movies_one[['name', 'weighted_average_vote']]

movies_one.rename(columns = {'weighted_average_vote':'average_role_score'}, inplace=True)

### Scores of Actors, Actresses, and Directors

In [19]:
actors_role = movies.groupby(['name']).sum() / movies.groupby(['name']).count()
actors_role = actors_role.reset_index()

In [20]:
actors_role = actors_role[['name', 'movie_title', 'role_actor', 'role_actress', 'role_director', 'weighted_average_vote']]
actress_role = actors_role
directors_role = actors_role

# Setting dataframes based on the dummified columns
actors_role = actors_role[actors_role['role_actor'] >= 1]
actress_role = actress_role[actress_role['role_actress'] >= 1]
directors_role = directors_role[directors_role['role_director'] >= 1]

In [21]:
# Create new dataframe of each individual actor and their average score
actors_role = actors_role[['name', 'weighted_average_vote']]
actors_role.rename(columns = {'weighted_average_vote':'actor_score'}, inplace=True)

# Create new dataframe of each individual actress and their average score
actress_role = actress_role[['name', 'weighted_average_vote']]
actress_role.rename(columns = {'weighted_average_vote':'actress_score'}, inplace=True)

# Create new dataframe of each individual director and their average score
directors_role = directors_role[['name', 'weighted_average_vote']]
directors_role.rename(columns = {'weighted_average_vote':'director_score'}, inplace=True)

In [22]:
# Visualizing each actor and their average score
actors_role

,name,actor_score
0,'Ducky' Louie,6.400000
1,'Weird Al' Yankovic,7.000000
2,50 Cent,5.166667
4,A. Michael Baldwin,6.200000
5,A.J. Buckley,4.700000
...,...,...
12432,Zakes Mokae,6.500000
12433,Zakk Wylde,6.700000
12437,Zane Holtz,5.000000
12438,Zane Pais,6.000000


In [23]:
# Visualizing each actress and their average score
actress_role

,name,actress_score
6,A.J. Cook,6.40
8,A.J. Langer,6.40
10,AJ Michalka,6.55
11,Aaliyah,6.10
34,Aarti Mann,6.60
...,...,...
12459,Zoë Bell,5.06
12460,Zoë Kravitz,6.75
12461,Zoë Lund,6.80
12462,Zulay Henao,5.90


In [24]:
# Visualizing each director and their average score
directors_role

,name,director_score
3,A. Edward Sutherland,6.1
7,A.J. Kparr,4.7
13,Aaron Blaise,6.8
17,Aaron Hann,6.0
18,Aaron Harvey,4.6
...,...,...
12430,Zackary Adler,5.7
12434,Zal Batmanglij,6.6
12435,Zalman King,4.7
12455,Zoltan Korda,7.5


### Merge Role Scores to DataFrame

In [25]:
# Merging movies with the average role scores.
final_movies = pd.merge(movies, movies_one, left_on = 'name', right_on = 'name')
final_movies = final_movies.groupby(['movie_title']).sum() / final_movies.groupby(['movie_title']).count()
final_movies = final_movies.reset_index()
final_movies.rename(columns = {'average_role_score':'casting_score'}, inplace=True)

In [26]:
# Merging average actors scores for each movie by movie
final_movies_actors = pd.merge(movies, actors_role, left_on = 'name', right_on = 'name')
final_movies_actors = final_movies_actors.groupby(['movie_title']).sum() / final_movies_actors.groupby(['movie_title']).count()
final_movies_actors = final_movies_actors.reset_index()

In [27]:
# Merging average actresses scores for each movie by movie
final_movies_actresses = pd.merge(movies, actress_role, left_on = 'name', right_on = 'name')
final_movies_actresses = final_movies_actresses.groupby(['movie_title']).sum() / final_movies_actresses.groupby(['movie_title']).count()
final_movies_actresses = final_movies_actresses.reset_index()

In [28]:
# Merging average directors scores for each movie by movie
final_movie_directors = pd.merge(movies, directors_role, left_on = 'name', right_on = 'name')
final_movie_directors = final_movie_directors.groupby(['movie_title']).sum() / final_movie_directors.groupby(['movie_title']).count()
final_movie_directors = final_movie_directors.reset_index()

In [29]:
# Used to merge cast, duration, and plot back to the dataframe
movies_details = movies[['movie_title', 'actors', 'duration', 'plot', 'tagline', 'popularity', 'budget', 'revenue']]

In [30]:
final_df = pd.merge(final_movies, movies_details, left_on = 'movie_title', right_on = 'movie_title')

In [31]:
# Rename columns
final_df.rename(columns = {'plot_y':'plot',
                             'duration_y':'duration',
                             'actors_y':'cast'}, inplace=True)

# Create total_score column which is an average of weighted_average_vote, us_voters_rating, mean_vote, and median_vote
final_df['total_score'] = (final_df['weighted_average_vote'] + final_df['us_voters_rating'] + final_df['mean_vote'] + final_df['median_vote']) / 4

# Dropping duplicates
final_df = final_df.drop_duplicates()

# Rounding casting_score
final_df['casting_score'] = final_df['casting_score'].round(3)

In [32]:
# Ensuring each directors, actors, and actresses dataframe only contains their score and I can merge on movie_title
final_movie_directors = final_movie_directors[['movie_title', 'director_score']]
final_movies_actors = final_movies_actors[['movie_title', 'actor_score']]
final_movies_actresses = final_movies_actresses[['movie_title', 'actress_score']]

final_df = pd.merge(final_df, final_movie_directors, left_on = 'movie_title', right_on = 'movie_title')
final_df = pd.merge(final_df, final_movies_actors, left_on = 'movie_title', right_on = 'movie_title')
final_df = pd.merge(final_df, final_movies_actresses, left_on = 'movie_title', right_on = 'movie_title')

final_df = final_df.round(3)

In [33]:
# Filling tagline where missing with Unknown
final_df['tagline_y'] = final_df['tagline_y'].fillna("Unknown")

In [34]:
# Found duplicate values. Dropping to ensure data is valid
final_df = final_df.drop_duplicates(subset = ['movie_title'])

In [35]:
# Converting year to integer (whole number) from float (decimal)
final_df['year'] = final_df['year'].astype('int64')

In [36]:
final_df.rename(columns = {'tagline_y':'tagline',
                           'popularity_y':'popularity',
                           'budget_y': 'budget',
                          'revenue_y': 'revenue'}, inplace=True)

In [37]:
final_df = final_df[(final_df['budget'] != 0) & (final_df['revenue'] != 0)]
final_df = final_df[final_df['year'] > 1970]

In [38]:
# Into the Night
final_df.at[1852, 'budget'] = 8000000

# Joyful Noise
final_df.at[1956, 'budget'] = 25000000
final_df.at[1956, 'revenue'] = 31158113

# Lost & Found
final_df.at[2143, 'budget'] = 30000000
final_df.at[2143, 'revenue'] = 6552255

# Tales from the Darkside: The Movie
final_df.at[3365, 'budget'] = 3500000
final_df.at[3365, 'revenue'] = 16324573

# The Prophecy
final_df.at[4087, 'budget'] = 8000000
final_df.at[4087, 'revenue'] = 16115878

# Bordello of Blood
final_df.at[572, 'revenue'] = 5781045

# Jason Goes to Hell: The Final Friday
final_df.at[1908, 'revenue'] = 15935068

# The Good Doctor
final_df.at[3721, 'revenue'] = 51017

In [39]:
final_df = final_df[final_df['revenue'] > 10000]

In [40]:
final_df['budget_adj'] = final_df['budget_adj'].astype(int)
final_df['revenue_adj'] = final_df['revenue_adj'].astype(int)

In [41]:
final_df['net_profit'] = final_df['revenue'] - final_df['budget']
final_df['profitable'] = [1 if x >= 0 else 0 for x in final_df['net_profit']]

### Sentiment Analysis of Plot and Tagline

In [42]:
plot_desc = final_df['plot'].tolist()

analyzer = SentimentIntensityAnalyzer()

def get_polarity_plot(plot_desc):
    polarity = []
    for post in plot_desc:
        vs = analyzer.polarity_scores(post)
        polarity.append(vs['compound'])
    return polarity

polarity = get_polarity_plot(plot_desc)

final_df['plot_sentiment'] = polarity

In [43]:
tagline_desc = final_df['tagline'].tolist()

analyzer_two = SentimentIntensityAnalyzer()

def get_polarity_tag(tagline_desc):
    polarity_tag = []
    for tag in tagline_desc:
        vs = analyzer_two.polarity_scores(tag)
        polarity_tag.append(vs['compound'])
    return polarity_tag

polarity_tag = get_polarity_tag(tagline_desc)

final_df['tagline_sentiment'] = polarity_tag

### Ordering of Columns in DataFrame

In [44]:
# Organizing columns
final_df = final_df[['movie_title', 'year', 'tagline', 'plot', 'cast', 'duration', 'Action',
                     'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Drama', 'Family',
                     'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 
                     'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western', 'avg_vote', 'total_votes', 
                     'us_voters_votes', 'votes', 'votes_1', 'votes_2', 'votes_3', 'votes_4', 
                     'votes_5', 'votes_6', 'votes_7', 'votes_8', 'votes_9', 'votes_10', 'popularity', 
                     'director_score', 'actor_score', 'actress_score','tagline_sentiment', 'plot_sentiment', 
                     'total_score', 'profitable', 'budget', 'revenue', 'budget_adj', 'revenue_adj',]]

In [45]:
final_df = final_df.dropna()
final_df = final_df.reset_index()
final_df.drop(columns = ['index'], inplace=True)

In [46]:
final_df

,movie_title,year,tagline,plot,cast,duration,Action,Adventure,Animation,Biography,...,actor_score,actress_score,tagline_sentiment,plot_sentiment,total_score,profitable,budget,revenue,budget_adj,revenue_adj
0,(500) Days of Summer,2009,It was almost like falling in love.,An offbeat romantic comedy about a woman who d...,"Joseph Gordon-Levitt, Zooey Deschanel, Geoffre...",95,0.0,0.0,0.0,0.0,...,6.746,6.308,0.7257,-0.1280,7.825,1,7500000,60722734,7623003,61718613
1,10 Things I Hate About You,1999,How do I loathe thee? Let me count the ways.,"A pretty, popular teenager can't go out on a d...","Heath Ledger, Julia Stiles, Joseph Gordon-Levi...",97,0.0,0.0,0.0,0.0,...,6.821,6.319,-0.4939,0.7184,7.275,1,16000000,53478166,20944845,70005745
2,10th & Wolf,2006,"The Intersection Where Family, Honor and Betra...",A former street tough returns to his Philadelp...,"James Marsden, Giovanni Ribisi, Brad Renfro, P...",107,0.0,0.0,0.0,0.0,...,6.258,5.975,-0.2263,-0.1280,6.575,0,8000000,143451,8653353,155166
3,12 Rounds,2009,Survive all 12,Detective Danny Fisher discovers his girlfrien...,"John Cena, Aidan Gillen, Ashley Scott, Steve H...",108,1.0,0.0,0.0,0.0,...,5.553,6.025,0.0000,0.8316,5.700,0,20000000,17280326,20328008,17563730
4,1408,2007,The only demons in room 1408 are those within ...,A man who specialises in debunking paranormal ...,"John Cusack, Tony Shalhoub, Len Cariou, Isiah ...",104,0.0,0.0,0.0,0.0,...,6.615,6.080,0.0000,-0.6486,6.875,1,25000000,94679598,26291714,99571558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,"Yours, Mine & Ours",2005,"18 kids, one house, no way.",A widowed Coast Guard Admiral and a widow hand...,"Dennis Quaid, Rene Russo, Sean Faris, Katija P...",88,0.0,0.0,0.0,0.0,...,5.838,6.627,-0.2960,0.1531,5.625,1,45000000,72028752,50245348,80424660
1579,Zero Effect,1998,The world's most private detective.,The world's greatest detective Daryl Zero aide...,"Bill Pullman, Ben Stiller, Ryan O'Neal, Kim Di...",116,0.0,0.0,0.0,0.0,...,6.409,6.900,0.0000,0.5093,7.050,0,5000000,1980338,6688477,2649089
1580,Zombieland,2009,This place is so dead,A shy student trying to reach his family in Oh...,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",88,0.0,1.0,0.0,0.0,...,6.550,6.665,-0.7573,0.3182,7.775,1,23600000,102391382,23987050,104070645
1581,Zookeeper,2011,Welcome to his jungle.,A group of zoo animals decide to break their c...,"Kevin James, Rosario Dawson, Leslie Bibb, Ken ...",102,0.0,0.0,0.0,0.0,...,5.639,6.115,0.4588,0.9011,5.150,1,80000000,169852759,77551843,164654932


### Export Final DataFrame

In [47]:
final_df.to_csv('./data/final_df.csv', index=False)